In [1]:
import pandas as pd
import numpy as np
from icecream import ic
import seaborn as sns
from matplotlib import pyplot as plt
sns.set()

In [2]:
source_filepath = '../datasets/oecd-population.csv'
target_filepath = '../plots'
oecd_pop_df = pd.read_csv(source_filepath)

In [3]:
source_dir = '../datasets/wits-imports-exports/'
data_file = lambda year: f'WITS-Partner_{year}.xlsx'
target_filepath = '../plots'

columns_of_interest = ['Partner Name', 'Year', 'Export (US$ Thousand)', 'Import (US$ Thousand)']
wits_imports_exports = pd.DataFrame(columns=columns_of_interest)

for year in range(1988, 2022):
    file_name = source_dir+data_file(year)
    ic(file_name)
    with pd.ExcelFile(file_name, engine='openpyxl') as xlsx:
       wits_imports_exports = pd.concat(
           [wits_imports_exports, pd.read_excel(xlsx, "Partner")[columns_of_interest]])
       
wits_imports_exports.rename(columns={'Partner Name': 'Country'}, inplace=True)
wits_imports_exports.to_csv('../datasets/wits-imports-exports.csv', index=False)
        

ic| file_name: '../datasets/wits-imports-exports/WITS-Partner_1988.xlsx'
/var/folders/pt/7c62stp91n90_kb8qmmqs5d00000gn/T/ipykernel_16611/220033432.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  wits_imports_exports = pd.concat(
ic| file_name: '../datasets/wits-imports-exports/WITS-Partner_1989.xlsx'
ic| file_name: '../datasets/wits-imports-exports/WITS-Partner_1990.xlsx'
ic| file_name: '../datasets/wits-imports-exports/WITS-Partner_1991.xlsx'
ic| file_name: '../datasets/wits-imports-exports/WITS-Partner_1992.xlsx'
ic| file_name: '../datasets/wits-imports-exports/WITS-Partner_1993.xlsx'
ic| file_name: '../datasets/wits-imports-exports/WITS-Partner_1994.xlsx'
ic| file_name: '../datasets/wits-imports-exports/WITS-Partner_1995.xlsx'
ic| 

In [4]:
wits_imports_exports = pd.read_csv('../datasets/wits-imports-exports.csv')
wits_imports_exports.head()

,Country,Year,Export (US$ Thousand),Import (US$ Thousand)
0,Aruba,1988,3498.10,328.49
1,Afghanistan,1988,213030.40,54459.52
2,Angola,1988,375527.89,370702.76
3,Anguila,1988,366.98,4.00
4,Albania,1988,30103.56,47709.30


In [5]:
wits_imports_exports[(wits_imports_exports['Country'] == 'China') & (wits_imports_exports['Year'] == 2021)]

,Country,Year,Export (US$ Thousand),Import (US$ Thousand)
7893,China,2021,2.419607e+09,3.383436e+09


In [12]:
len(wits_imports_exports['Country'].unique())

265

In [6]:
# wits_imports_exports[wits_imports_exports['Country'] == 'United States'].index[0]
np.where(wits_imports_exports['Country'].unique() == 'United States')[0][0]

183

In [7]:
selected_country = 'Japan'
country_imports_exports = wits_imports_exports.query('Country == @selected_country').drop('Country', axis=1)
country_imports_exports.head()

,Year,Export (US$ Thousand),Import (US$ Thousand)
94,1988,3.195074e+07,5.212943e+07
301,1989,7.080756e+07,9.527899e+07
512,1990,7.889416e+07,1.105882e+08
724,1991,1.291768e+08,2.238075e+08
943,1992,1.455659e+08,2.602507e+08


In [8]:
thousand_to_million_round2 = lambda n: np.round(n / 1000, 2)

country_imports_exports['Export in US$ Millions'] = country_imports_exports['Export (US$ Thousand)'].apply(thousand_to_million_round2)
country_imports_exports.drop('Export (US$ Thousand)', axis=1, inplace=True)

country_imports_exports['Import in US$ Millions'] = country_imports_exports['Import (US$ Thousand)'].apply(thousand_to_million_round2)
country_imports_exports.drop('Import (US$ Thousand)', axis=1, inplace=True)

# country_imports_exports['Trade Balance'] = country_imports_exports['Export in US$ Millions'] - country_imports_exports['Import in US$ Millions']
country_imports_exports.head()

,Year,Export in US$ Millions,Import in US$ Millions
94,1988,31950.74,52129.43
301,1989,70807.56,95278.99
512,1990,78894.16,110588.18
724,1991,129176.82,223807.53
943,1992,145565.90,260250.70


In [9]:
country_trade_balance = pd.DataFrame(columns=['Year', 'Export or Import', 'Value in US$ Millions'])
for year in country_imports_exports['Year']:
    export_value = country_imports_exports.query('Year == @year')['Export in US$ Millions'].iloc[0]
    import_value = -country_imports_exports.query('Year == @year')['Import in US$ Millions'].iloc[0]
    trade_value = export_value + import_value
    
    export_row = {'Year': year, 'Export or Import': 'Export', 'Value in US$ Millions': export_value, 'Trade Balance in US$ Millions':trade_value}
    import_row = {'Year': year, 'Export or Import': 'Import', 'Value in US$ Millions': import_value, 'Trade Balance in US$ Millions':trade_value}

    country_trade_balance = pd.concat([country_trade_balance, pd.DataFrame([export_row, import_row])])

country_trade_balance.head()

/var/folders/pt/7c62stp91n90_kb8qmmqs5d00000gn/T/ipykernel_16611/3078453641.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  country_trade_balance = pd.concat([country_trade_balance, pd.DataFrame([export_row, import_row])])


,Year,Export or Import,Value in US$ Millions,Trade Balance in US$ Millions
0,1988,Export,31950.74,-20178.69
1,1988,Import,-52129.43,-20178.69
0,1989,Export,70807.56,-24471.43
1,1989,Import,-95278.99,-24471.43
0,1990,Export,78894.16,-31694.02
